In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.signal as signal
from pathlib import Path
from filterpy.kalman import KalmanFilter, IMMEstimator
from filterpy.common import Q_discrete_white_noise, kinematic_kf
from scipy.linalg import block_diag

import os
import sys
sys.path.append("../src")

from fast_open_data import open_data_filter
from data_conversions import transition_matrix, sep_modo

In [2]:
def estimates(list_voluntary):
    f_name = 'C:/Users/User/OneDrive/TCC/ema_motion_analysis_imu/data/data_angle/Transitions/'
    sensor = ['_S_truth', '_R_truth', '_L_truth']
    #list_voluntary = [5, 6, 8, 13, 15, 16, 21, 23, 24, 25, 27, 28, 29, 30, 31, 34, 36, 38, 39, 40, 41, 
    #                   45, 47, 49, 51, 52, 55, 57, 60, 65, 66, 67, 68, 69, 71, 75, 78, 80, 81, 86, 87]
    est_manual = []

    for i in list_voluntary:
        for j in range(len(sensor)):
            data_path = f_name + str(i) + sensor[j]
            est_manual.append(np.loadtxt(data_path, delimiter=';'))
    return est_manual

In [4]:
def extract_metrics(est_manual):
    
    freq_amostragem = 120
    
    t = []
    dif_temp_sit_SR = []
    dif_temp_sit_SL = []
    dif_temp_rise_SR = []
    dif_temp_rise_SL = []
    seg_sit_SR = []
    seg_sit_SL = []
    seg_rise_SR = []
    seg_rise_SL = []
    est_IMM_S = []
    est_IMM_R = []
    est_IMM_L = []
    seq_IMM_S = []
    seq_IMM_R = []
    seq_IMM_L = []

    for k in range(len(list_voluntary)):
        
        S, R, L = open_data_filter(list_voluntary[k], key = 'S1_Synched')
        # Criando vetor de tempo
        t = np.linspace(0, len(S) / freq_amostragem, len(S))
        
        modelo_S = Kalman_Filter_S(S)
        modelo_R = Kalman_Filter_Leg(R)
        modelo_L = Kalman_Filter_Leg(L)
        
        #modelo_S, modelo_R, modelo_L = Kalman_Filter(S, R, L)
        
        
        est_S = []
        est_R = []
        est_L = []
        seq_S = []
        seq_R = []
        seq_L = []
        for i in range(len(modelo_S)-1):
            if (modelo_S[i] != modelo_S[i+1]):
                est_S.append(i)
                seq_S.append(modelo_S[i])
                
            if (modelo_R[i] != modelo_R[i+1]):
                est_R.append(i)
                seq_R.append(modelo_R[i])
                
            if (modelo_L[i] != modelo_L[i+1]):
                est_L.append(i)
                seq_L.append(modelo_L[i])
        
        if seq_S == [1, 2, 3, 1, 2, 3] and seq_R == [1, 2, 3, 4] and seq_L == [1, 2, 3, 4] and len(est_S) == 6 and len(est_R) == 4 and len(est_L) == 4 :
            seq_IMM_S.append(np.array(seq_S))
            seq_IMM_R.append(np.array(seq_R))
            seq_IMM_L.append(np.array(seq_L))
            est_IMM_S.append(np.array(est_S))
            est_IMM_R.append(np.array(est_R))
            est_IMM_L.append(np.array(est_L))
                    
            est_manual_S = est_manual[k*3]
            est_manual_R = est_manual[k*3 + 1]
            est_manual_L = est_manual[k*3 + 2]
        
            est_sit_manual_SR, est_sit_manual_SL = est_sit1(est_manual_S, est_manual_R, est_manual_L)
            est_sit_IMM_SR, est_sit_IMM_SL = est_sit1(est_IMM_S[0], est_IMM_R[0], est_IMM_L[0])
            est_rise_manual_SR, est_rise_manual_SL = est_rise1(est_manual_S, est_manual_R, est_manual_L)
            est_rise_IMM_SR, est_rise_IMM_SL = est_rise1(est_IMM_S[0], est_IMM_R[0], est_IMM_L[0])
                
            dif_sit_SR = calc_dif(est_sit_manual_SR, est_sit_IMM_SR, t)
            dif_sit_SL = calc_dif(est_sit_manual_SL, est_sit_IMM_SL, t)
            dif_rise_SR = calc_dif(est_rise_manual_SR, est_rise_IMM_SR, t)
            dif_rise_SL = calc_dif(est_rise_manual_SL, est_rise_IMM_SL, t)
        
            seg_time_sit_SR = calc_seg_time(est_sit_manual_SR, est_sit_IMM_SR, t)
            seg_time_sit_SL = calc_seg_time(est_sit_manual_SL, est_sit_IMM_SL, t)
            seg_time_rise_SR = calc_seg_time(est_rise_manual_SR, est_rise_IMM_SR, t)
            seg_time_rise_SL = calc_seg_time(est_rise_manual_SL, est_rise_IMM_SL, t)
        
            dif_temp_sit_SR.append(dif_sit_SR) 
            dif_temp_sit_SL.append(dif_sit_SL) 
            dif_temp_rise_SR.append(dif_rise_SR) 
            dif_temp_rise_SL.append(dif_rise_SL)
        
            seg_sit_SR.append(seg_time_sit_SR)
            seg_sit_SL.append(seg_time_sit_SL)
            seg_rise_SR.append(seg_time_rise_SR)
            seg_rise_SL.append(seg_time_rise_SL)  
            
            #print('Voluntário ' + str(list_voluntary[k]) + ' OK!')
        
        #else:
            #print('Voluntário ' + str(list_voluntary[k]) + ' NÃO OK!')
    
    dif_temp_sit_SR = np.array(dif_temp_sit_SR)
    dif_temp_sit_SL = np.array(dif_temp_sit_SL)
    dif_temp_rise_SR = np.array(dif_temp_rise_SR)
    dif_temp_rise_SL = np.array(dif_temp_rise_SL)
    
    seg_sit_SR = np.array(seg_sit_SR)
    seg_sit_SL = np.array(seg_sit_SL)
    seg_rise_SR = np.array(seg_rise_SR)
    seg_rise_SL = np.array(seg_rise_SL)

    return dif_temp_sit_SR, dif_temp_sit_SL, dif_temp_rise_SR, dif_temp_rise_SL, seg_sit_SR, seg_sit_SL, seg_rise_SR, seg_rise_SL

In [5]:
def calc_seg_time(est_manual, est_IMM, t):
    
    seg_time_fase2_manual = t[est_manual[1]] - t[est_manual[0]]
    seg_time_fase3_manual = t[est_manual[2]] - t[est_manual[1]]
    
    seg_time_fase2_IMM = t[est_IMM[1]] - t[est_IMM[0]]
    seg_time_fase3_IMM = t[est_IMM[2]] - t[est_IMM[1]]
    
    seg_time = np.c_[seg_time_fase2_manual, seg_time_fase3_manual, seg_time_fase2_IMM, seg_time_fase3_IMM]
    
    
    return seg_time

In [6]:
def calc_dif(est_manual, est_IMM, t):
    dif_temp = t[est_manual] - t[est_IMM]
    return dif_temp

In [7]:
def est_rise1(est_S, est_R, est_L):
    
    sep = int((est_S[2] + est_S[3])/2)
    
    est_rise_SR = np.array([est_S[3]-sep, est_S[4]-sep, min(est_S[5], est_R[3])-sep])
    est_rise_SR = np.sort(est_rise_SR)
    est_rise_SL = np.array([est_S[3]-sep, est_S[4]-sep, min(est_S[5], est_L[3])-sep])
    est_rise_SL = np.sort(est_rise_SL)
    
    est_rise_SR = est_rise_SR.astype('int')
    est_rise_SL = est_rise_SL.astype('int')
    
    return  est_rise_SR, est_rise_SL  

In [8]:
def est_rise(est_S, est_R, est_L):
    
    sep = int((est_S[2] + est_S[3])/2)
    
    est_rise_SR = np.array([est_S[3]-sep, est_S[4]-sep, max(est_S[5], est_R[3])-sep])
    est_rise_SR = np.sort(est_rise_SR)
    est_rise_SL = np.array([est_S[3]-sep, est_S[4]-sep, max(est_S[5], est_L[3])-sep])
    est_rise_SL = np.sort(est_rise_SL)
    
    est_rise_SR = est_rise_SR.astype('int')
    est_rise_SL = est_rise_SL.astype('int')
    
    return  est_rise_SR, est_rise_SL   

In [9]:
def est_sit1(est_S, est_R, est_L):

    est_sit_SR = np.array([max(est_S[0], est_R[0]), est_S[1], est_S[2]])
    est_sit_SR = np.sort(est_sit_SR)
    est_sit_SL = np.array([max(est_S[0], est_L[0]), est_S[1], est_S[2]])
    est_sit_SL = np.sort(est_sit_SL)
    
    est_sit_SR = est_sit_SR.astype('int')
    est_sit_SL = est_sit_SL.astype('int')
    
    return  est_sit_SR, est_sit_SL   

In [10]:
def est_sit(est_S, est_R, est_L):

    est_sit_SR = np.array([min(est_S[0], est_R[0]), est_S[1], est_S[2]])
    est_sit_SR = np.sort(est_sit_SR)
    est_sit_SL = np.array([min(est_S[0], est_L[0]), est_S[1], est_S[2]])
    est_sit_SL = np.sort(est_sit_SL)
    
    est_sit_SR = est_sit_SR.astype('int')
    est_sit_SL = est_sit_SL.astype('int')
    
    return  est_sit_SR, est_sit_SL   

In [11]:
def Kalman_Filter_Leg(data):
    
    transitions = np.array([[0.98586777, 0.01413223, 0.        , 0.        ],
                            [0.        , 0.98545455, 0.01454545, 0.        ],
                            [0.        , 0.        , 0.98649123, 0.01350877],
                            [0.01540541, 0.        , 0.        , 0.98459459]])
    
    kf1 = KalmanFilter(2, 1)
    kf2 = KalmanFilter(2, 1)
    kf3 = KalmanFilter(2, 1)
    kf4 = KalmanFilter(2, 1)

    kf1.x = np.array([95, 1]).T
    kf2.x = np.array([95, 1]).T
    kf3.x = np.array([20, 1]).T
    kf4.x = np.array([20, 1]).T

    kf1.F = np.array([[1, 0.005],[0, 1]])
    kf2.F = np.array([[1, -0.4],[0, 1]])
    kf3.F = np.array([[1, 0.005],[0, 1]])
    kf4.F = np.array([[1, 0.4],[0, 1]])

    kf1.H = np.array([[1, 0]])
    kf2.H = np.array([[1, 0]])
    kf3.H = np.array([[1, 0]])
    kf4.H = np.array([[1, 0]])

    kf1.R = 5.040
    kf2.R = 5.100
    kf3.R = 4.980
    kf4.R = 4.990

    kf1.Q = np.array([[0.02, 0],[0, 0.005]])
    kf2.Q = np.array([[0.02, 0],[0, 0.005]])
    kf3.Q = np.array([[0.02, 0],[0, 0.005]])
    kf4.Q = np.array([[0.02, 0],[0, 0.005]])

    filters = [kf1, kf2, kf3, kf4]
    mu = [0.97, 0.01, 0.01, 0.01]  # each filter is equally likely at the start
    M = np.array(transitions)
    imm = IMMEstimator(filters, mu, M)
    
    z = np.c_[data]
    modelo = []

    for i, z in enumerate(z):
        
        # perform predict/update cycle
        imm.predict()
        imm.update(z)

        estado = np.where(imm.mu == max(imm.mu))[0][0]
        modelo.append(estado)
    
    modelo = np.array(modelo)
    modelo += 1
    
    return modelo

In [12]:
def Kalman_Filter_S(data):
    
    transitions = np.array([[0.987, 0.013, 0.],
                            [0., 0.987, 0.013],
                            [0.013, 0., 0.987]])
    
    kf1 = KalmanFilter(2, 1)
    kf2 = KalmanFilter(2, 1)
    kf3 = KalmanFilter(2, 1)

    kf1.x = np.array([70, 1]).T
    kf2.x = np.array([70, 1]).T
    kf3.x = np.array([30, 1]).T

    kf1.F = np.array([[1, 0.01],[0, 1]])
    kf2.F = np.array([[1, -0.4],[0, 1]])
    kf3.F = np.array([[1, 0.4],[0, 1]])

    kf1.H = np.array([[1, 0]])
    kf2.H = np.array([[1, 0]])
    kf3.H = np.array([[1, 0]])

    kf1.R = 181.8
    kf2.R = 183.5
    kf3.R = 182.5

    kf1.Q = np.array([[0.02, 0],[0, 0.005]])
    kf2.Q = np.array([[0.02, 0],[0, 0.005]])
    kf3.Q = np.array([[0.02, 0],[0, 0.005]])

    filters = [kf1, kf2, kf3]
    mu = [0.98, 0.01, 0.01]  # each filter is equally likely at the start
    M = np.array(transitions)
    imm = IMMEstimator(filters, mu, M)
    
    z = np.c_[data]
    modelo = []

    for i, z in enumerate(z):
   

        # perform predict/update cycle
        imm.predict()
        imm.update(z)

        estado = np.where(imm.mu == max(imm.mu))[0][0]
        modelo.append(estado)
    
    modelo = np.array(modelo)
    modelo += 1
    
    return modelo

In [14]:
list_voluntary = [5, 6, 8, 13, 15, 16, 21, 23, 24, 25, 27, 28, 29, 30, 31, 34, 36, 38, 39, 40, 41, 
                  45, 47, 49, 51, 52, 55, 57, 60, 65, 66, 67, 68, 69, 71, 75, 78, 80, 81, 86, 87]

In [16]:
est_manual = estimates(list_voluntary)
t_sit_SR, t_sit_SL, t_rise_SR, t_rise_SL, f_sit_SR, f_sit_SL, f_rise_SR, f_rise_SL = extract_metrics(est_manual)

In [186]:
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        SENTAR (TRONCO E PERNA DIREITA)                                            ')
print('')
fase_1_2_sit_manual_SR = np.mean(abs(t_sit_SR[:,0]))
print('Média do erro na detecção da transição FaseI/FaseII', round(fase_1_2_sit_manual_SR, 4))
fase_2_3_sit_manual_SR = np.mean(abs(t_sit_SR[:,1]))
print('Média do erro na detecção da transição FaseII/FaseIII', round(fase_2_3_sit_manual_SR, 4))
fase_3_4_sit_manual_SR = np.mean(abs(t_sit_SR[:,2]))
print('Média do erro na detecção da transição FaseIII/FaseIV', round(fase_3_4_sit_manual_SR, 4))
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        SENTAR (TRONCO E PERNA ESQUERDA)                                           ')
print('')
fase_1_2_sit_manual_SL = np.mean(abs(t_sit_SL[:,0]))
print('Média do erro na detecção da transição FaseI/FaseII', round(fase_1_2_sit_manual_SL, 4))
fase_2_3_sit_manual_SL = np.mean(abs(t_sit_SL[:,1]))
print('Média do erro na detecção da transição FaseII/FaseIII', round(fase_2_3_sit_manual_SL, 4))
fase_3_4_sit_manual_SL = np.mean(abs(t_sit_SL[:,2]))
print('Média do erro na detecção da transição FaseIII/FaseIV', round(fase_3_4_sit_manual_SL, 4))
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        LEVANTAR (TRONCO E PERNA DIREITA)                                          ')
print('')
fase_1_2_rise_manual_SR = np.mean(abs(t_rise_SR[:,0]))
print('Média do erro na detecção da transição FaseI/FaseII', round(fase_1_2_rise_manual_SR, 4))
fase_2_3_rise_manual_SR = np.mean(abs(t_rise_SR[:,1]))
print('Média do erro na detecção da transição FaseII/FaseIII', round(fase_2_3_rise_manual_SR, 4))
fase_3_4_rise_manual_SR = np.mean(abs(t_rise_SR[:,2]))
print('Média do erro na detecção da transição FaseIII/FaseIV', round(fase_3_4_rise_manual_SR, 4))
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        LEVANTAR (TRONCO E PERNA ESQUERDA)                                         ')
print('')
fase_1_2_rise_manual_SL = np.mean(abs(t_rise_SL[:,0]))
print('Média do erro na detecção da transição FaseI/FaseII', round(fase_1_2_rise_manual_SL, 4))
fase_2_3_rise_manual_SL = np.mean(abs(t_rise_SL[:,1]))
print('Média do erro na detecção da transição FaseII/FaseIII', round(fase_2_3_rise_manual_SL, 4))
fase_3_4_rise_manual_SL = np.mean(abs(t_rise_SL[:,2]))
print('Média do erro na detecção da transição FaseIII/FaseIV', round(fase_3_4_rise_manual_SL, 4))

-------------------------------------------------------------------------------------------------------------------
                                        SENTAR (TRONCO E PERNA DIREITA)                                            

Média do erro na detecção da transição FaseI/FaseII 0.4255
Média do erro na detecção da transição FaseII/FaseIII 0.4037
Média do erro na detecção da transição FaseIII/FaseIV 0.3396
-------------------------------------------------------------------------------------------------------------------
                                        SENTAR (TRONCO E PERNA ESQUERDA)                                           

Média do erro na detecção da transição FaseI/FaseII 0.5044
Média do erro na detecção da transição FaseII/FaseIII 0.4037
Média do erro na detecção da transição FaseIII/FaseIV 0.3396
-------------------------------------------------------------------------------------------------------------------
                                        LEVANTAR (TRONCO

In [187]:
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        SENTAR (TRONCO E PERNA DIREITA)                                            ')
print('')
tempo_sit_fase_2_manual_SR = np.mean(f_sit_SR[:,:,0])
print('Tempo médio para fase II - Segmentação Manual', 
      round(tempo_sit_fase_2_manual_SR, 4), 'segundos')
tempo_sit_fase_2_DTW_SR = np.mean(f_sit_SR[:,:,2])
print('Tempo médio para fase II - Segmentação -> IMM', 
      round(tempo_sit_fase_2_DTW_SR, 4), 'segundos')

tempo_sit_fase_3_manual_SR = np.mean(f_sit_SR[:,:,1])
print('Tempo médio para fase III - Segmentação Manual', 
      round(tempo_sit_fase_3_manual_SR, 4), 'segundos')
tempo_sit_fase_3_DTW_SR = np.mean(f_sit_SR[:,:,3])
print('Tempo médio para fase III - Segmentação -> IMM', 
      round(tempo_sit_fase_3_DTW_SR, 4), 'segundos')
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        SENTAR (TRONCO E PERNA ESQUERDA)                                           ')
print('')
tempo_sit_fase_2_manual_SL = np.mean(f_sit_SL[:,:,0])
print('Tempo médio para fase II - Segmentação Manual', 
      round(tempo_sit_fase_2_manual_SL, 4), 'segundos')
tempo_sit_fase_2_DTW_SL = np.mean(f_sit_SL[:,:,2])
print('Tempo médio para fase II - Segmentação -> IMM', 
      round(tempo_sit_fase_2_DTW_SL, 4), 'segundos')

tempo_sit_fase_3_manual_SL = np.mean(f_sit_SL[:,:,1])
print('Tempo médio para fase III - Segmentação Manual', 
      round(tempo_sit_fase_3_manual_SL, 4), 'segundos')
tempo_sit_fase_3_DTW_SL = np.mean(f_sit_SL[:,:,3])
print('Tempo médio para fase III - Segmentação -> IMM', 
      round(tempo_sit_fase_3_DTW_SL, 4), 'segundos')
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        LEVANTAR (TRONCO E PERNA DIREITA)                                          ')
print('')
tempo_rise_fase_2_manual_SR = np.mean(f_rise_SR[:,:,0])
print('Tempo médio para fase II - Segmentação Manual', 
      round(tempo_rise_fase_2_manual_SR, 4), 'segundos')
tempo_rise_fase_2_DTW_SR = np.mean(f_rise_SR[:,:,2])
print('Tempo médio para fase II - Segmentação -> IMM', 
      round(tempo_rise_fase_2_DTW_SR, 4), 'segundos')

tempo_rise_fase_3_manual_SR = np.mean(f_rise_SR[:,:,1])
print('Tempo médio para fase III - Segmentação Manual', 
      round(tempo_rise_fase_3_manual_SR, 4), 'segundos')
tempo_rise_fase_3_DTW_SR = np.mean(f_rise_SR[:,:,3])
print('Tempo médio para fase III - Segmentação -> DTW', 
      round(tempo_rise_fase_3_DTW_SR, 4), 'segundos')
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        LEVANTAR (TRONCO E PERNA ESQUERDA)                                         ')
print('')
tempo_rise_fase_2_manual_SL = np.mean(f_rise_SL[:,:,0])
print('Tempo médio para fase II - Segmentação Manual', 
      round(tempo_rise_fase_2_manual_SL, 4), 'segundos')
tempo_rise_fase_2_DTW_SL = np.mean(f_rise_SL[:,:,2])
print('Tempo médio para fase II - Segmentação -> IMM', 
      round(tempo_rise_fase_2_DTW_SL, 4), 'segundos')

tempo_rise_fase_3_manual_SL = np.mean(f_rise_SL[:,:,1])
print('Tempo médio para fase III - Segmentação Manual', 
      round(tempo_rise_fase_3_manual_SL, 4), 'segundos')
tempo_rise_fase_3_DTW_SL = np.mean(f_rise_SL[:,:,3])
print('Tempo médio para fase III - Segmentação -> IMM', 
      round(tempo_rise_fase_3_DTW_SL, 4), 'segundos')

-------------------------------------------------------------------------------------------------------------------
                                        SENTAR (TRONCO E PERNA DIREITA)                                            

Tempo médio para fase II - Segmentação Manual 0.2477 segundos
Tempo médio para fase II - Segmentação -> IMM 0.0001 segundos
Tempo médio para fase III - Segmentação Manual 0.2201 segundos
Tempo médio para fase III - Segmentação -> IMM 0.0001 segundos
-------------------------------------------------------------------------------------------------------------------
                                        SENTAR (TRONCO E PERNA ESQUERDA)                                           

Tempo médio para fase II - Segmentação Manual 0.2771 segundos
Tempo médio para fase II - Segmentação -> IMM 0.0001 segundos
Tempo médio para fase III - Segmentação Manual 0.2201 segundos
Tempo médio para fase III - Segmentação -> IMM 0.0001 segundos
----------------------------------